In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.font_manager import FontProperties
from collections import Counter
from wordcloud import WordCloud
import numpy as np

# --- 参数与资源设置 ---
N = 10 # 可以随时调整此数值
TOP_K = 100+N  # 新增：需要输出的高频词数量

# 定义您的字体文件路径
font_path = '/kaggle/input/nssfc-2025/simkai.ttf'
# 定义您的数据文件路径
csv_path = '/kaggle/input/nssfc-2025/nssfc-fenci.csv'

# 在这里添加您不希望统计的词语
# 1. 研究元词汇与方法论相关
research_meta_stopwords = [
    '影响', '体系', '视域', '机理', '效应', '构建', '建构', '理论', '实践', 
    '对策', '视角', '背景', '基于', '模式', '应用', '问题', '评估', '测度', 
    '关系', '形成', '重构', '逻辑', '形态', '系统', '模型', '演化', '演变', 
    '经验', '资料', '史料', '要素', '研究', '机制', '路径', '及其'
]

# 2. 高度泛化的背景词
generalized_context_stopwords = [
    '中国', '社会', '我国', '国家', '时代', '当代', '时期', '历史', 
    '世纪', '1949', '近代'
]

# 3. 抽象的过程、目标与修饰词
abstract_process_stopwords = [
    '策略', '提升', '优化', '整理', '驱动', '赋能', '建设', '实现', '应对', 
    '促进', '困境', '动态', '识别', '效能', '交融'
]

custom_stopwords = set(research_meta_stopwords+generalized_context_stopwords+abstract_process_stopwords)

# 加载字体文件
font_prop = FontProperties(fname=font_path)

# --- 数据加载与计算 ---
df = pd.read_csv(csv_path)
# 注意：根据您上传的文件，列名应为'项目名称分词列'
df.dropna(subset=['课题名称分词'], inplace=True)
words_in_docs = [set(text.split()) for text in df['课题名称分词']]
all_words = [word for doc in words_in_docs for word in doc]
doc_freq = Counter(all_words)

# 从词频统计中移除停用词
for word in custom_stopwords:
    if word in doc_freq:
        del doc_freq[word]


top_k_words = doc_freq.most_common(TOP_K)
print(f"前{TOP_K}个高频词（按文档频率）:")
for w, f in top_k_words:
    print(f"（{w}，{f}）")


# --- 绘制前N个高频词的条形图 ---
top_n_words = doc_freq.most_common(N)
words, freqs = zip(*reversed(top_n_words))

# 2. 为条形图创建漂亮的渐变色
cmap = cm.get_cmap('plasma')
norm = colors.Normalize(vmin=min(freqs), vmax=max(freqs))
bar_colors = cmap(norm(freqs))

# 创建图表
plt.figure(figsize=(10, 6))
plt.barh(words, freqs, color=bar_colors)

# 设置中文标题和标签
plt.title(f'全部项目名称前 {N} 个高频词 (文档频率)', fontproperties=font_prop, fontsize=18)
plt.xlabel('项目数量 (文档频率)', fontproperties=font_prop, fontsize=15)
plt.ylabel('词语', fontproperties=font_prop, fontsize=15)

# 3. 将字体和稍大的字号应用到坐标轴刻度上
plt.xticks(fontproperties=font_prop, fontsize=12)
plt.yticks(fontproperties=font_prop, fontsize=12)

# 在每个条形上显示具体的数字
for index, value in enumerate(freqs):
    plt.text(value, index, f' {value}', va='center', fontproperties=font_prop, fontsize=10)

plt.tight_layout()
plt.savefig(f'top_{N}_words_barchart_overall.png', dpi=300)
print(f"优化后的前 {N} 个高频词条形图已保存为 top_{N}_words_barchart_overall.png")

# --- 生成并保存词云图 ---
wordcloud_freq = doc_freq.copy()
# 剔除已在条形图中显示的前N个词
# for word, freq in top_n_words:
#     del wordcloud_freq[word]

x, y = np.ogrid[:800, :800]
mask = (x - 400)**2 + (y - 400)**2 > 400**2
mask = 255 * mask.astype(int)

wordcloud = WordCloud(
    font_path=font_path,
    background_color='white',
    width=800,
    height=800,
    mask=mask,
    max_words=2000,
    colormap='viridis',
    prefer_horizontal=1.0 # 4. 设置所有词语水平显示，不旋转
).generate_from_frequencies(wordcloud_freq)

plt.figure(figsize=(10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.savefig('wordcloud_overall.png', dpi=300)
print("优化后的中文词云图已保存为 wordcloud_overall.png")

前110个高频词（按文档频率）:
（治理，601）
（智能，432）
（人工，350）
（数字，350）
（政策，350）
（风险，326）
（发展，310）
（民族，295）
（创新，292）
（文化，283）
（协同，283）
（企业，262）
（地区，223）
（技术，219）
（数据，201）
（传播，186）
（产业，184）
（经济，176）
（中华，176）
（生态，173）
（制度，162）
（国际，159）
（价值，152）
（安全，150）
（转型，145）
（文学，142）
（融合，134）
（服务，132）
（共同体，129）
（韧性，125）
（文献，125）
（网络，124）
（数智，120）
（生成，119）
（城市，112）
（知识，112）
（乡村，109）
（质量，106）
（体育，105）
（政治，103）
（全球，101）
（美国，101）
（意识，100）
（认知，97）
（西南，97）
（叙事，97）
（思想，95）
（法律，94）
（现代化，93）
（行为，92）
（生成式，91）
（空间，90）
（西部，88）
（共产党，87）
（农业，85）
（语言，84）
（农村，83）
（边疆，81）
（金融，79）
（就业，79）
（干预，79）
（话语，78）
（教育，78）
（公共，74）
（资本，74）
（传统，73）
（科技，73）
（汉语，73）
（平台，70）
（健康，69）
（基层，68）
（绿色，68）
（市场，68）
（区域，67）
（青年，67）
（战略，65）
（数字化，64）
（生产力，63）
（消费，63）
（资源，63）
（环境，63）
（供应链，62）
（人口，62）
（清代，62）
（青少年，62）
（改革，60）
（制造业，60）
（群体，60）
（认同，59）
（社区，59）
（文明，58）
（合作，58）
（日本，58）
（新质，57）
（小说，57）
（新时代，56）
（哲学，56）
（政府，56）
（冲击，56）
（作用，56）
（能力，55）
（破解，55）
（生产，54）
（抗战，54）
（贸易，54）
（支持，54）
（模态，54）
（现代，53）
（伦理，53）
（互动，53）


/tmp/ipykernel_37/593508995.py:70: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('plasma')


优化后的前 10 个高频词条形图已保存为 top_10_words_barchart_overall.png
优化后的中文词云图已保存为 wordcloud_overall.png


<div align="center">

| 涉及学科 | 样本数 | 平均值 | 中位数 | 众数 | 标准差 | 四分位距（IQR） |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 全部项目 | 5767 | 21.8 | 22.0 | 22 | 5.37 | 8.0 |
| 管理学 | 590 | 25.95 | 26.0 | 28 | 3.99 | 6.0 |
| 应用经济学 | 509 | 25.3 | 26.0 | 27 | 3.75 | 5.0 |
| 理论经济学 | 191 | 25.5 | 26.0 | 25 | 3.61 | 4.5 |
| 信息资源管理与文献学 | 145 | 21.48 | 22.0 | 22 | 4.59 | 5.0 |
| 新闻学与传播学 | 211 | 22.85 | 23.0 | 21 | 4.13 | 5.0 |

</div>